In [32]:
import pandas as pd
from rdt.transformers import ClusterBasedNormalizer

In [33]:
df = pd.read_csv('insurance_claims.csv')

In [34]:
df.head()

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,_c39
0,328,48,521585,2014-10-17,OH,250/500,1000,1406.91,0,466132,...,YES,71610,6510,13020,52080,Saab,92x,2004,Y,NaN
1,228,42,342868,2006-06-27,IN,250/500,2000,1197.22,5000000,468176,...,?,5070,780,780,3510,Mercedes,E400,2007,Y,NaN
2,134,29,687698,2000-09-06,OH,100/300,2000,1413.14,5000000,430632,...,NO,34650,7700,3850,23100,Dodge,RAM,2007,N,NaN
3,256,41,227811,1990-05-25,IL,250/500,2000,1415.74,6000000,608117,...,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,Y,NaN
4,228,44,367455,2014-06-06,IL,500/1000,1000,1583.91,6000000,610706,...,NO,6500,1300,650,4550,Accura,RSX,2009,N,NaN


## transform continuous columns
* genarete named tuple containing information of a column

In [35]:
from collections import namedtuple
SpanInfo = namedtuple('SpanInfo', ['dim', 'activation_fn'])
ColumnTransformInfo = namedtuple(
    'ColumnTransformInfo', [
        'column_name', 'column_type', 'transform', 'output_info', 'output_dimensions'
    ]
)

In [36]:
def _fit_continuous(data):
        """Train Bayesian GMM for continuous columns.
        Args:
            data (pd.DataFrame):
                A dataframe containing a column.
        Returns:
            namedtuple:
                A ``ColumnTransformInfo`` object.
        """
        column_name = data.columns[0]
        gm = ClusterBasedNormalizer(model_missing_values=True, max_clusters=min(len(data), 10))
        gm.fit(data, column_name)
        num_components = sum(gm.valid_component_indicator)

        return ColumnTransformInfo(
            column_name=column_name, column_type='continuous', transform=gm,
            output_info=[SpanInfo(1, 'tanh'), SpanInfo(num_components, 'softmax')],
            output_dimensions=1 + num_components)

In [37]:
# data (pd.DataFrame):A dataframe containing a column.
data = df[['total_claim_amount']]
_fit_continuous(data)

ColumnTransformInfo(column_name='total_claim_amount', column_type='continuous', transform=ClusterBasedNormalizer(model_missing_values=True), output_info=[SpanInfo(dim=1, activation_fn='tanh'), SpanInfo(dim=6, activation_fn='softmax')], output_dimensions=7)

In [38]:
# data (pd.DataFrame):A dataframe containing a column.
data = df[['total_claim_amount']]
type(data), len(data)


(pandas.core.frame.DataFrame, 1000)

In [39]:
column_name = data.columns[0]
column_name

'total_claim_amount'

In [40]:
gm = ClusterBasedNormalizer(model_missing_values=True, max_clusters=min(len(data), 10))

In [41]:
gm.fit(data, column_name)

In [42]:
gm.get_output_columns(), gm.columns

(['total_claim_amount.normalized', 'total_claim_amount.component'],
 ['total_claim_amount'])

In [43]:
gm.valid_component_indicator

array([ True,  True,  True,  True,  True,  True, False, False, False,
       False])

In [44]:
num_components = sum(gm.valid_component_indicator)
num_components

6

In [45]:
transformed_data = gm.transform(data)
transformed_data

,total_claim_amount.normalized,total_claim_amount.component
0,0.043611,2.0
1,-0.037903,0.0
2,-0.472925,1.0
3,0.126648,4.0
4,0.041942,0.0
...,...,...
995,0.420494,1.0
996,0.570246,2.0
997,0.193973,3.0
998,-0.308193,2.0


## transform discrete columns

In [46]:
from rdt.transformers import OneHotEncoder

In [47]:
def _fit_discrete(data):
        """Fit one hot encoder for discrete column.
        Args:
            data (pd.DataFrame):
                A dataframe containing a column.
        Returns:
            namedtuple:
                A ``ColumnTransformInfo`` object.
        """
        column_name = data.columns[0]
        ohe = OneHotEncoder()
        ohe.fit(data, column_name)
        num_categories = len(ohe.dummies)

        return ColumnTransformInfo(
            column_name=column_name, column_type='discrete', transform=ohe,
            output_info=[SpanInfo(num_categories, 'softmax')],
            output_dimensions=num_categories)

In [48]:
data = df[['auto_make']]
column_name = data.columns[0]

In [49]:
ohe = OneHotEncoder()
ohe.fit(data, column_name)

In [50]:
ohe.dummies

['Saab',
 'Mercedes',
 'Dodge',
 'Chevrolet',
 'Accura',
 'Nissan',
 'Audi',
 'Toyota',
 'Ford',
 'Suburu',
 'BMW',
 'Jeep',
 'Honda',
 'Volkswagen']

In [51]:
num_categories = len(ohe.dummies)

In [52]:
transformed_data = ohe.transform(data)
transformed_data

,auto_make.value0,auto_make.value1,auto_make.value2,auto_make.value3,auto_make.value4,auto_make.value5,auto_make.value6,auto_make.value7,auto_make.value8,auto_make.value9,auto_make.value10,auto_make.value11,auto_make.value12,auto_make.value13
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,0,0,1,0
996,0,0,0,0,0,0,0,0,0,0,0,0,0,1
997,0,0,0,0,0,0,0,0,0,1,0,0,0,0
998,0,0,0,0,0,0,1,0,0,0,0,0,0,0


## fit function
* aggregate the transform information(namedtuples) of all columns in raw data in `column_transform_info_list`, also the output information of all columns(including output dimension and activation function_ as namedtuples ) are stored in `output_info_list`. also the output dimension of transformed data

In [53]:
def fit(raw_data, discrete_columns=()):
        """Fit the ``DataTransformer``.
        Fits a ``ClusterBasedNormalizer`` for continuous columns and a
        ``OneHotEncoder`` for discrete columns.
        This step also counts the #columns in matrix data and span information.
        """
        output_info_list = []
        output_dimensions = 0
        dataframe = True

        column_raw_dtypes = raw_data.infer_objects().dtypes
        column_transform_info_list = []
        for column_name in raw_data.columns:
            if column_name in discrete_columns:
                column_transform_info = _fit_discrete(raw_data[[column_name]])
            else:
                column_transform_info = _fit_continuous(raw_data[[column_name]])

            output_info_list.append(column_transform_info.output_info)
            output_dimensions += column_transform_info.output_dimensions
            column_transform_info_list.append(column_transform_info)

In [54]:
df_new = df[['injury_claim', 'property_claim', 'vehicle_claim', 'auto_make', 'auto_model', 'auto_year']]
raw_data = df_new

In [55]:
column_raw_dtypes = raw_data.infer_objects().dtypes

In [56]:
column_transform_info_list = []
discrete_columns = ['auto_make', 'auto_model', 'auto_year']
output_info_list = []
output_dimensions = 0

In [57]:
for column_name in raw_data.columns:
    if column_name in discrete_columns:
        column_transform_info = _fit_discrete(raw_data[[column_name]])
    else:
        column_transform_info = _fit_continuous(raw_data[[column_name]])

    output_info_list.append(column_transform_info.output_info)
    output_dimensions += column_transform_info.output_dimensions
    column_transform_info_list.append(column_transform_info)


In [58]:
output_info_list

[[SpanInfo(dim=1, activation_fn='tanh'),
  SpanInfo(dim=3, activation_fn='softmax')],
 [SpanInfo(dim=1, activation_fn='tanh'),
  SpanInfo(dim=5, activation_fn='softmax')],
 [SpanInfo(dim=1, activation_fn='tanh'),
  SpanInfo(dim=5, activation_fn='softmax')],
 [SpanInfo(dim=14, activation_fn='softmax')],
 [SpanInfo(dim=39, activation_fn='softmax')],
 [SpanInfo(dim=21, activation_fn='softmax')]]

In [59]:
output_dimensions

90

In [60]:
column_transform_info_list

[ColumnTransformInfo(column_name='injury_claim', column_type='continuous', transform=ClusterBasedNormalizer(model_missing_values=True), output_info=[SpanInfo(dim=1, activation_fn='tanh'), SpanInfo(dim=3, activation_fn='softmax')], output_dimensions=4),
 ColumnTransformInfo(column_name='property_claim', column_type='continuous', transform=ClusterBasedNormalizer(model_missing_values=True), output_info=[SpanInfo(dim=1, activation_fn='tanh'), SpanInfo(dim=5, activation_fn='softmax')], output_dimensions=6),
 ColumnTransformInfo(column_name='vehicle_claim', column_type='continuous', transform=ClusterBasedNormalizer(model_missing_values=True), output_info=[SpanInfo(dim=1, activation_fn='tanh'), SpanInfo(dim=5, activation_fn='softmax')], output_dimensions=6),
 ColumnTransformInfo(column_name='auto_make', column_type='discrete', transform=OneHotEncoder(), output_info=[SpanInfo(dim=14, activation_fn='softmax')], output_dimensions=14),
 ColumnTransformInfo(column_name='auto_model', column_type='d

In [61]:
data_dim = output_dimensions

## build generator

In [62]:
#define residual layer
import tensorflow as tf
import math
class ResidualLayer(tf.keras.layers.Layer):
    def __init__(self,input_dim, output_dim):
        super(ResidualLayer, self).__init__()
        bound = 1/math.sqrt(input_dim)
        w_init = tf.keras.initializers.RandomUniform(minval=-bound, maxval=bound)
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_dim, output_dim), dtype="float32"),
            trainable=True,
        )
        b_init = tf.keras.initializers.RandomUniform(minval=-bound, maxval=bound)
        self.b = tf.Variable(
            initial_value=b_init(shape=(output_dim,), dtype="float32"), trainable=True
        )
    def call(self, inputs):
        x = tf.matmul(inputs, self.w) + self.b
        x = tf.keras.layers.BatchNormalization(epsilon=1e-5, momentum=0.9)(x)
        x = tf.keras.layers.ReLU()(x)
        out = tf.concat([x, inputs],1)
        return out

2023-06-07 23:33:12.396374: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [63]:
embedding_dim = 128 + 74
generator_dim = [256, 256]

In [64]:
with tf.device('/gpu:1'):
    inp_gen = tf.keras.layers.Input(shape=(embedding_dim,))
    input_dim = embedding_dim
    x = inp_gen
    for output_dim in generator_dim:
       x = ResidualLayer(input_dim, output_dim)(x)
       input_dim += output_dim
    x = tf.keras.layers.Dense(units = data_dim)(x)
    generator = tf.keras.models.Model(inp_gen, x)


2023-06-07 23:33:17.491700: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-06-07 23:33:17.493337: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-06-07 23:33:17.546275: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 47.45GiB deviceMemoryBandwidth: 625.94GiB/s
2023-06-07 23:33:17.546486: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:af:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 47.45GiB deviceMemoryBandwidth: 625.94GiB/s
2023-06-07 23:33:17.546511: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-06-07 23:3

In [65]:
generator.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 202)]             0         
_________________________________________________________________
residual_layer (ResidualLaye (None, 458)               51968     
_________________________________________________________________
residual_layer_1 (ResidualLa (None, 714)               117504    
_________________________________________________________________
dense (Dense)                (None, 90)                64350     
Total params: 233,822
Trainable params: 233,822
Non-trainable params: 0
_________________________________________________________________


## apply activation function to the output of generator

In [66]:
# gumble softmax function
import tensorflow_probability as tfp
def _gumbel_softmax(logits, tau=1.0, hard=False, dim=-1):
        """Samples from the Gumbel-Softmax distribution
        :cite:`maddison2016concrete`, :cite:`jang2016categorical` and
        optionally discretizes.
        Parameters
        ----------
        logits: tf.Tensor
            Un-normalized log probabilities.
        tau: float, default=1.0
            Non-negative scalar temperature.
        hard: bool, default=False
            If ``True``, the returned samples will be discretized as
            one-hot vectors, but will be differentiated as soft samples.
        dim: int, default=1
            The dimension along which softmax will be computed.
        Returns
        -------
        tf.Tensor
            Sampled tensor of same shape as ``logits`` from the
            Gumbel-Softmax distribution. If ``hard=True``, the returned samples
            will be one-hot, otherwise they will be probability distributions
            that sum to 1 across ``dim``.
        """

        gumbel_dist = tfp.distributions.Gumbel(loc=0, scale=1)
        gumbels = gumbel_dist.sample(tf.shape(logits))
        gumbels = (logits + gumbels) / tau
        output = tf.nn.softmax(gumbels, dim)

        if hard:
            index = tf.math.reduce_max(output, 1, keepdims=True)
            output_hard = tf.cast(tf.equal(output, index), output.dtype)
            output = tf.stop_gradient(output_hard - output) + output
        return output

In [67]:
def _apply_activate(data):
        """Apply proper activation function to the output of the generator."""
        data_t = []
        st = 0
        for column_info in transformer.output_info_list:
            for span_info in column_info:
                if span_info.activation_fn == 'tanh':
                    ed = st + span_info.dim
                    data_t.append(tf.math.tanh(data[:, st:ed]))
                    st = ed
                elif span_info.activation_fn == 'softmax':
                    ed = st + span_info.dim
                    transformed = _gumbel_softmax(data[:, st:ed], tau=0.2)
                    data_t.append(transformed)
                    st = ed
                else:
                    raise ValueError(f'Unexpected activation function {span_info.activation_fn}.')

        return tf.concat(data_t, axis=1)

In [68]:
import tensorflow as tf
with tf.device('/gpu:1'):
    BATCH_SIZE = 10
    noise_dim = embedding_dim
    noise = tf.random.normal((BATCH_SIZE, noise_dim))
    noise.shape

In [69]:
with tf.device('/gpu:1'):
   data = generator(noise)
   data

2023-06-07 23:33:32.782988: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


In [70]:
import tensorflow as tf
data_t = []
st = 0
for column_info in output_info_list:
    for span_info in column_info:
        if span_info.activation_fn == 'tanh':
            ed = st + span_info.dim
            data_t.append(tf.math.tanh(data[:, st:ed]))
            st = ed
        elif span_info.activation_fn == 'softmax':
            ed = st + span_info.dim
            transformed = _gumbel_softmax(data[:, st:ed], tau=0.2)
            data_t.append(transformed)
            st = ed
        else:
            raise ValueError(f'Unexpected activation function {span_info.activation_fn}.')

In [71]:
data_t

[<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[-0.46928915],
        [ 0.95056313],
        [-0.82112604],
        [ 0.7702087 ],
        [ 0.9398631 ],
        [ 0.40016192],
        [ 0.567068  ],
        [ 0.6374217 ],
        [ 0.14365566],
        [ 0.7339894 ]], dtype=float32)>,
 <tf.Tensor: shape=(10, 3), dtype=float32, numpy=
 array([[5.9801809e-05, 9.9994016e-01, 1.9737163e-10],
        [1.0000000e+00, 2.8790041e-17, 1.2520658e-18],
        [1.9458004e-05, 5.3363975e-02, 9.4661653e-01],
        [7.2719109e-01, 2.6197481e-01, 1.0834107e-02],
        [3.3437785e-02, 2.8935727e-01, 6.7720497e-01],
        [6.5587270e-01, 1.5700830e-04, 3.4397033e-01],
        [4.6598888e-12, 1.5961944e-12, 1.0000000e+00],
        [3.6696589e-01, 3.4121264e-02, 5.9891284e-01],
        [4.1588614e-06, 9.9957281e-01, 4.2295986e-04],
        [4.9342336e-08, 1.5303436e-01, 8.4696561e-01]], dtype=float32)>,
 <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[ 0.06272586],
        [ 0

In [72]:
tf.concat(data_t, axis=1)

<tf.Tensor: shape=(10, 90), dtype=float32, numpy=
array([[-4.69289154e-01,  5.98018087e-05,  9.99940157e-01,
         1.97371633e-10,  6.27258644e-02,  5.68970136e-05,
         1.03327108e-03,  5.31241655e-01,  4.67668146e-01,
         6.95091629e-08, -4.25103873e-01,  9.92195845e-01,
         2.47599257e-08,  3.10022733e-05,  5.88513649e-06,
         7.76728429e-03,  3.16408150e-05,  6.52995823e-07,
         2.76290348e-06,  1.03650741e-07,  2.96262659e-09,
         4.22370831e-05,  2.09763311e-04,  1.25429665e-06,
         9.99704659e-01,  4.50257756e-12,  4.44563775e-10,
         2.33435645e-12,  2.36961633e-08,  6.91570585e-06,
         4.94935454e-15,  8.56819726e-10,  2.26630398e-11,
         2.76565935e-08,  1.32608841e-10,  7.04967226e-14,
         4.82928855e-13,  4.34302844e-10,  1.01794996e-07,
         1.75669996e-15,  2.87612078e-12,  3.01356440e-11,
         9.64627933e-10,  7.87640135e-13,  3.83290811e-13,
         2.32915208e-02,  2.64640458e-12,  1.54112851e-11,
      

## convert a column of raw data to appropriate format for discriminator

In [73]:
# convert a column of data to appropriate format for discriminator

def _transform_continuous(column_transform_info, data):
    column_name = data.columns[0]
    flattened_column = data[column_name].to_numpy().flatten()
    data = data.assign(**{column_name: flattened_column})
    gm = column_transform_info.transform
    transformed = gm.transform(data)

    #  Converts the transformed data to the appropriate output format.
    #  The first column (ending in '.normalized') stays the same,
    #  but the lable encoded column (ending in '.component') is one hot encoded.
    output = np.zeros((len(transformed), column_transform_info.output_dimensions))
    output[:, 0] = transformed[f'{column_name}.normalized'].to_numpy()
    index = transformed[f'{column_name}.component'].to_numpy().astype(int)
    output[np.arange(index.size), index + 1] = 1.0

    return output

In [74]:
column_transform_info = column_transform_info_list[0]
column_name = column_transform_info.column_name
data = raw_data[[column_name]]
column_name = data.columns[0]
column_name

'injury_claim'

In [75]:
data

,injury_claim
0,6510
1,780
2,7700
3,6340
4,1300
...,...
995,17440
996,18080
997,7500
998,5220


In [76]:
data[column_name]

0       6510
1        780
2       7700
3       6340
4       1300
       ...  
995    17440
996    18080
997     7500
998     5220
999      460
Name: injury_claim, Length: 1000, dtype: int64

In [77]:
flattened_column = data[column_name].to_numpy().flatten()
flattened_column

array([ 6510,   780,  7700,  6340,  1300,  6410, 21450,  9380,  2770,
        4700,  7910, 17680,  4710,  1120,  4200, 10520,  5790, 14160,
        6630,  6040,     0,     0, 17880,  8180,  7080, 16500,  1640,
        1040,  7760, 14100, 12600,  7460,  3310, 14020, 10800, 10620,
        6020,  1230, 12460, 10940,  8000, 16180,  5740,  5680, 11280,
        6890,     0,  6280,   810, 15320, 16360,  1320,   430, 12820,
         480, 15780,     0,   300,  7310, 11440, 15440,  7380,  5630,
       11420,  6570, 13720, 13800,     0, 12460,   860, 12420,  6810,
        3010,  9520,  9540, 11380, 14900, 10860,  1240, 14440, 10160,
         660,   750,  1180,  5540,  5830, 11400, 11680,   940,     0,
       10300,  8940,   590,  5890, 17040,  1260,  6630,  7210,   900,
         700, 15860, 10560,     0,   330, 15580,   480,  6650,  7420,
       10860,  5540,  7470, 14000, 14740, 14430,  1180,  4770,  9320,
        6400, 18000, 13240, 13860, 12760,     0,  8570,  7000,  3300,
        5760,   330,

In [78]:
data = data.assign(**{column_name: flattened_column})
data

,injury_claim
0,6510
1,780
2,7700
3,6340
4,1300
...,...
995,17440
996,18080
997,7500
998,5220


In [79]:
#  Converts the transformed data to the appropriate output format.
#  The first column (ending in '.normalized') stays the same,
#  but the lable encoded column (ending in '.component') is one hot encoded.
gm = column_transform_info.transform
transformed = gm.transform(data)
transformed

,injury_claim.normalized,injury_claim.component
0,0.090276,2.0
1,0.015343,0.0
2,0.303125,2.0
3,0.059869,2.0
4,0.189583,0.0
...,...,...
995,0.427383,1.0
996,0.480425,1.0
997,0.267352,2.0
998,-0.140459,2.0


In [80]:
len(transformed)

1000

In [81]:
column_transform_info.output_dimensions

4

In [82]:
import numpy as np
output = np.zeros((len(transformed), column_transform_info.output_dimensions))
output

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [83]:
transformed[f'{column_name}.normalized'].to_numpy()

array([ 0.0902761 ,  0.015343  ,  0.30312498,  0.05986911,  0.18958275,
        0.07238964,  0.75972111, -0.24060727, -0.78842587, -0.23346884,
        0.34068655,  0.44727388, -0.2316802 ,  0.12926899, -0.32290115,
       -0.14612721, -0.03850642,  0.15554597,  0.11173985, -0.51741727,
       -0.24601662, -0.24601662,  0.46384933,  0.38897999,  0.19222892,
        0.34947873,  0.30350874,  0.10246288,  0.31385686,  0.15057334,
        0.02625747,  0.26019747, -0.48209065,  0.14394316, -0.12292158,
       -0.13783948,  0.00263244,  0.1661274 ,  0.01465465, -0.11131876,
        0.35678436,  0.32295801, -0.04744965, -0.05818153, -0.0831405 ,
        0.15824465, -0.24601662,  0.04913724,  0.02539529,  0.25168358,
        0.33787592,  0.19628428, -0.10193375,  0.04449046, -0.08517993,
        0.28980711, -0.24601662, -0.14549369,  0.23336778, -0.06988014,
        0.26162885,  0.24588831, -0.06712476, -0.07153768,  0.10100797,
        0.11907998,  0.12571016, -0.24601662,  0.01465465,  0.04

In [84]:
output[:, 0] = transformed[f'{column_name}.normalized'].to_numpy()
output

array([[ 0.0902761 ,  0.        ,  0.        ,  0.        ],
       [ 0.015343  ,  0.        ,  0.        ,  0.        ],
       [ 0.30312498,  0.        ,  0.        ,  0.        ],
       ...,
       [ 0.26735206,  0.        ,  0.        ,  0.        ],
       [-0.14045925,  0.        ,  0.        ,  0.        ],
       [-0.09188146,  0.        ,  0.        ,  0.        ]])

In [85]:
index = transformed[f'{column_name}.component'].to_numpy().astype(int)
index

array([2, 0, 2, 2, 0, 2, 1, 1, 1, 2, 2, 1, 2, 0, 2, 1, 2, 1, 2, 1, 0, 0,
       1, 2, 2, 1, 0, 0, 2, 1, 1, 2, 2, 1, 1, 1, 2, 0, 1, 1, 2, 1, 2, 2,
       1, 2, 0, 2, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 2, 1, 1, 2, 2, 1, 2, 1,
       1, 0, 1, 0, 1, 2, 2, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 2, 2, 1, 1,
       0, 0, 1, 1, 0, 2, 1, 0, 2, 2, 0, 0, 1, 1, 0, 0, 1, 0, 2, 2, 1, 2,
       2, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 0, 1, 2, 2, 2, 0, 1, 2, 1, 2,
       2, 2, 1, 1, 0, 2, 1, 1, 1, 0, 0, 2, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1,
       2, 0, 2, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 2, 1, 1, 0, 1,
       2, 1, 1, 0, 1, 1, 1, 1, 2, 1, 0, 0, 1, 1, 0, 2, 2, 0, 2, 2, 0, 0,
       2, 0, 0, 2, 0, 2, 2, 2, 1, 2, 2, 0, 0, 0, 1, 1, 2, 2, 2, 0, 1, 1,
       1, 0, 2, 2, 1, 2, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2,
       0, 1, 0, 2, 2, 1, 0, 2, 2, 2, 0, 2, 2, 1, 1, 2, 0, 2, 0, 2, 2, 2,
       2, 0, 1, 1, 2, 1, 1, 0, 1, 1, 1, 2, 2, 1, 2, 1, 2, 0, 0, 1, 1, 1,
       2, 0, 2, 0, 2, 2, 1, 1, 2, 1, 1, 0, 0, 2, 0,

In [86]:
index.size

1000

In [87]:
output[np.arange(index.size), index + 1] = 1.0
output

array([[ 0.0902761 ,  0.        ,  0.        ,  1.        ],
       [ 0.015343  ,  1.        ,  0.        ,  0.        ],
       [ 0.30312498,  0.        ,  0.        ,  1.        ],
       ...,
       [ 0.26735206,  0.        ,  0.        ,  1.        ],
       [-0.14045925,  0.        ,  0.        ,  1.        ],
       [-0.09188146,  1.        ,  0.        ,  0.        ]])

In [88]:
def _transform_discrete(column_transform_info, data):
    ohe = column_transform_info.transform
    return ohe.transform(data).to_numpy()

In [89]:
column_transform_info = column_transform_info_list[3]
column_name = column_transform_info.column_name
data = raw_data[[column_name]]
data

,auto_make
0,Saab
1,Mercedes
2,Dodge
3,Chevrolet
4,Accura
...,...
995,Honda
996,Volkswagen
997,Suburu
998,Audi


In [90]:
ohe = column_transform_info.transform
ohe.transform(data).to_numpy()

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [91]:
def _synchronous_transform(raw_data, column_transform_info_list):
        """Take a Pandas DataFrame and transform columns synchronous.
        Outputs a list with Numpy arrays.
        """
        column_data_list = []
        for column_transform_info in column_transform_info_list:
            column_name = column_transform_info.column_name
            data = raw_data[[column_name]]
            if column_transform_info.column_type == 'continuous':
                column_data_list.append(_transform_continuous(column_transform_info, data))
            else:
                column_data_list.append(_transform_discrete(column_transform_info, data))

        return column_data_list

In [92]:
column_data_list = []
for column_transform_info in column_transform_info_list:
    column_name = column_transform_info.column_name
    data = raw_data[[column_name]]
    if column_transform_info.column_type == 'continuous':
        column_data_list.append(_transform_continuous(column_transform_info, data))
    else:
        column_data_list.append(_transform_discrete(column_transform_info, data))

In [93]:
column_data_list

[array([[ 0.0902761 ,  0.        ,  0.        ,  1.        ],
        [ 0.015343  ,  1.        ,  0.        ,  0.        ],
        [ 0.30312498,  0.        ,  0.        ,  1.        ],
        ...,
        [ 0.26735206,  0.        ,  0.        ,  1.        ],
        [-0.14045925,  0.        ,  0.        ,  1.        ],
        [-0.09188146,  1.        ,  0.        ,  0.        ]]),
 array([[ 0.14012159,  0.        ,  1.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.0108117 ,  1.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [-0.41557301,  0.        ,  0.        ,  1.        ,  0.        ,
          0.        ],
        ...,
        [ 0.28285664,  0.        ,  0.        ,  1.        ,  0.        ,
          0.        ],
        [-0.1534227 ,  0.        ,  0.        ,  1.        ,  0.        ,
          0.        ],
        [ 0.05825112,  1.        ,  0.        ,  0.        ,  0.        ,
          0.        ]]),
 array([[ 0.1804

In [94]:
def transform(raw_data):
    """Take raw data and output a matrix data."""
    column_data_list = _synchronous_transform(raw_data, _column_transform_info_list)
    return np.concatenate(column_data_list, axis=1).astype(float)

In [95]:
column_data_list = _synchronous_transform(raw_data, column_transform_info_list)
output_of_transform = np.concatenate(column_data_list, axis=1).astype(float)
output_of_transform.shape

(1000, 90)

In [96]:
output_of_transform

array([[ 0.0902761 ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.015343  ,  1.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.30312498,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.26735206,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.14045925,  0.        ,  0.        , ...,  1.        ,
         0.        ,  0.        ],
       [-0.09188146,  1.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

## DataSampler

In [97]:
output_info_list

[[SpanInfo(dim=1, activation_fn='tanh'),
  SpanInfo(dim=3, activation_fn='softmax')],
 [SpanInfo(dim=1, activation_fn='tanh'),
  SpanInfo(dim=5, activation_fn='softmax')],
 [SpanInfo(dim=1, activation_fn='tanh'),
  SpanInfo(dim=5, activation_fn='softmax')],
 [SpanInfo(dim=14, activation_fn='softmax')],
 [SpanInfo(dim=39, activation_fn='softmax')],
 [SpanInfo(dim=21, activation_fn='softmax')]]

In [98]:
def is_discrete_column(column_info):
    return (len(column_info) == 1 and column_info[0].activation_fn == 'softmax')

In [99]:
output_info_list

[[SpanInfo(dim=1, activation_fn='tanh'),
  SpanInfo(dim=3, activation_fn='softmax')],
 [SpanInfo(dim=1, activation_fn='tanh'),
  SpanInfo(dim=5, activation_fn='softmax')],
 [SpanInfo(dim=1, activation_fn='tanh'),
  SpanInfo(dim=5, activation_fn='softmax')],
 [SpanInfo(dim=14, activation_fn='softmax')],
 [SpanInfo(dim=39, activation_fn='softmax')],
 [SpanInfo(dim=21, activation_fn='softmax')]]

In [100]:
data = output_of_transform
output_info = output_info_list

In [101]:
st = 16
j=0
np.nonzero(data[:, st + j])[0]

array([  0,   5,   9,  12,  58,  78,  90,  98, 100, 101, 106, 108, 113,
       129, 154, 157, 167, 195, 204, 218, 228, 238, 240, 246, 252, 276,
       338, 359, 362, 370, 390, 392, 398, 431, 463, 476, 477, 492, 499,
       542, 554, 558, 565, 577, 589, 598, 599, 619, 623, 636, 642, 663,
       676, 677, 683, 686, 687, 689, 699, 712, 732, 758, 773, 794, 805,
       809, 841, 849, 854, 876, 877, 891, 895, 896, 899, 904, 907, 930,
       959, 979])

* for each discrete column we construct `rid_by_cat` list. so in this example we construct three `rid_by_cat` list, and add these three lists to `_rid_by_cat_cols` list. Each `rid_by_cat` contains a number of lists, one list for each category in the related discrete column. In each list we have the row ids that the value of discrete column is equal to the related category. 

In [102]:
# Store the row id for each category in each discrete column.
# For example _rid_by_cat_cols[a][b] is a list of all rows with the
# a-th discrete column equal value b.
_rid_by_cat_cols = []

# Compute _rid_by_cat_cols
st = 0
for column_info in output_info:
    if is_discrete_column(column_info):
        span_info = column_info[0]
        ed = st + span_info.dim

        rid_by_cat = []
        for j in range(span_info.dim):
            rid_by_cat.append(np.nonzero(data[:, st + j])[0])
        _rid_by_cat_cols.append(rid_by_cat)
        st = ed
    else:
        st += sum([span_info.dim for span_info in column_info])
assert st == data.shape[1]

In [103]:
len(_rid_by_cat_cols[0])

14

In [104]:
# Prepare an interval matrix for efficiently sample conditional vector
max_category = max([ column_info[0].dim for column_info in output_info if is_discrete_column(column_info)], default=0)

In [105]:
max_category

39

In [106]:
n_discrete_columns = sum([1 for column_info in output_info if is_discrete_column(column_info)])

In [107]:
n_discrete_columns

3

In [108]:
_discrete_column_cond_st = np.zeros(n_discrete_columns, dtype='int32')
_discrete_column_n_category = np.zeros(n_discrete_columns, dtype='int32')
_discrete_column_category_prob = np.zeros((n_discrete_columns, max_category))
_n_discrete_columns = n_discrete_columns
_n_categories = sum([column_info[0].dim for column_info in output_info if is_discrete_column(column_info)])


In [109]:
_n_categories

74

In [110]:
log_frequency = True

In [122]:
category_freq = np.sum(data[:, 30:69], axis=0)
 

In [123]:
np.sum(category_freq)

1000.0

In [111]:
st = 0
current_id = 0
current_cond_st = 0
for column_info in output_info:
    if is_discrete_column(column_info):
        span_info = column_info[0]
        ed = st + span_info.dim
        category_freq = np.sum(data[:, st:ed], axis=0)
        if log_frequency:
            category_freq = np.log(category_freq + 1)
        category_prob = category_freq / np.sum(category_freq)
        _discrete_column_category_prob[current_id, :span_info.dim] = category_prob
        _discrete_column_cond_st[current_id] = current_cond_st
        _discrete_column_n_category[current_id] = span_info.dim
        current_cond_st += span_info.dim
        current_id += 1
        st = ed
    else:
        st += sum([span_info.dim for span_info in column_info])

In [112]:
_discrete_column_category_prob

array([[0.0733736 , 0.06995417, 0.0733736 , 0.07252801, 0.07069638,
        0.07295615, 0.07093662, 0.07117346, 0.07163729, 0.0733736 ,
        0.07163729, 0.07045262, 0.06721082, 0.07069638, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.02662307, 0.02634563, 0.02991919, 0.02544961, 0.02027943,
        0.02634563, 0.02740138, 0.02764467, 0.02662307, 0.02634563,
        0.02876009, 0.02479037, 0.02876009, 0.02854924, 0.02544961,
        0.02764467, 0.02407111, 0.02407111, 0.02662307, 0.02544961,
        0.02407111, 0.02327981, 0.02327981, 0.02973743, 0.0219211 ,
        0.02512686, 0.02479037, 0.0278807 , 0.02479037, 0.02407111,
        0.02575971, 0.02086535, 0.02240042, 0.02715036, 0.0

In [113]:
 _discrete_column_cond_st

array([ 0, 14, 53], dtype=int32)

In [114]:
 _discrete_column_n_category

array([14, 39, 21], dtype=int32)

In [115]:
def _random_choice_prob_index(self, discrete_column_id):
    probs = self._discrete_column_category_prob[discrete_column_id]
    r = np.expand_dims(np.random.rand(probs.shape[0]), axis=1)
    return (probs.cumsum(axis=1) > r).argmax(axis=1)

In [116]:
# what is discrete_column_id in the function _random_choice_prob_index?
batch = 5
discrete_column_id = np.random.choice(np.arange(_n_discrete_columns), batch)
discrete_column_id

array([2, 1, 2, 0, 1])

In [117]:
probs = _discrete_column_category_prob[discrete_column_id]
probs

array([[0.04530002, 0.04875581, 0.04674642, 0.04828344, 0.0485219 ,
        0.04728042, 0.04753896, 0.04467013, 0.04804026, 0.04898536,
        0.04618813, 0.04828344, 0.04943196, 0.04898536, 0.04921069,
        0.04701632, 0.04964931, 0.04618813, 0.04560325, 0.04728042,
        0.04804026, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.02662307, 0.02634563, 0.02991919, 0.02544961, 0.02027943,
        0.02634563, 0.02740138, 0.02764467, 0.02662307, 0.02634563,
        0.02876009, 0.02479037, 0.02876009, 0.02854924, 0.02544961,
        0.02764467, 0.02407111, 0.02407111, 0.02662307, 0.02544961,
        0.02407111, 0.02327981, 0.02327981, 0.02973743, 0.0219211 ,
        0.02512686, 0.02479037, 0.0278807 , 0.02479037, 0.02407111,
        0.02575971, 0.02086535, 0.02240042, 0.02715036, 0.0

In [87]:
probs.shape

(5, 39)

In [88]:
np.random.rand(probs.shape[0])

array([0.73525242, 0.89662611, 0.40810784, 0.04870378, 0.47171214])

In [89]:
r = np.expand_dims(np.random.rand(probs.shape[0]), axis=1)
r

array([[0.79121127],
       [0.01022469],
       [0.11835301],
       [0.52365145],
       [0.48105196]])

In [90]:
probs.cumsum(axis=1) > r

array([[False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False,  True,  True,  True,  True,  True,  True,
         True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True],
       [False,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True],
       [False, False, False, False, False, False, False,  

In [91]:
(probs.cumsum(axis=1) > r).argmax(axis=1)

array([30,  0,  1,  7, 18])

In [92]:
cond = np.zeros((batch, _n_categories), dtype='float32')

In [93]:
category_id_in_col = (probs.cumsum(axis=1) > r).argmax(axis=1)
category_id_in_col

array([30,  0,  1,  7, 18])

In [94]:
category_id = (_discrete_column_cond_st[discrete_column_id] + category_id_in_col)
category_id

array([44,  0,  1,  7, 32])

In [95]:
cond[np.arange(batch), category_id] = 1

In [96]:
cond.shape

(5, 74)

In [97]:
mask = np.zeros((batch, _n_discrete_columns), dtype='float32')
mask[np.arange(batch), discrete_column_id] = 1
mask

array([[0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]], dtype=float32)

In [98]:
discrete_column_id, category_id_in_col

(array([1, 0, 0, 0, 1]), array([30,  0,  1,  7, 18]))

In [99]:
perm = np.arange(batch)
np.random.shuffle(perm)

In [100]:
output_info

[[SpanInfo(dim=1, activation_fn='tanh'),
  SpanInfo(dim=3, activation_fn='softmax')],
 [SpanInfo(dim=1, activation_fn='tanh'),
  SpanInfo(dim=5, activation_fn='softmax')],
 [SpanInfo(dim=1, activation_fn='tanh'),
  SpanInfo(dim=5, activation_fn='softmax')],
 [SpanInfo(dim=14, activation_fn='softmax')],
 [SpanInfo(dim=39, activation_fn='softmax')],
 [SpanInfo(dim=21, activation_fn='softmax')]]

In [1]:
def cross_entropy_loss(self, data, c, m, output_info):
        """Compute the cross entropy loss on the fixed discrete column."""
        loss = []
        st = 0
        st_c = 0
        for column_info in output_info:
            for span_info in column_info:
                if len(column_info) != 1 or span_info.activation_fn != 'softmax':
                    # not discrete column
                    st += span_info.dim
                else:
                    ed = st + span_info.dim
                    ed_c = st_c + span_info.dim
                    labels=tf.math.argmax(c[:, st_c:ed_c], 1)
                    labels = tf.reshape(labels, [-1,1])
                    logits=data[:, st:ed]
                    tmp = tf.nn.softmax_cross_entropy_with_logits(
                        labels,
                        logits)
                    loss.append(tmp)
                    st = ed
                    st_c = ed_c

        loss = tf.stack(loss, axis=1)
        m1 = tf.cast(m, dtype=tf.float32)
        return tf.reduce_mean(loss * m1)

## generator loss

In [2]:
from data_transformer import DataTransformer
from data_sampler import DataSampler
from ctgan import CTGAN

import pandas as pd
with tf.device('/gpu:1'):
   
     df = pd.read_csv('insurance_claims.csv')
     raw_data = df[['injury_claim', 'property_claim', 'vehicle_claim', 'auto_make', 'auto_model', 'auto_year']]

     transformer = DataTransformer()
     transformer.fit(raw_data, discrete_columns=('auto_make', 'auto_model', 'auto_year'))
     data_t = transformer.transform(raw_data)      #matrix of transformed data
     output_info = transformer.output_info_list

     log_frequency = True
     sampler = DataSampler(data_t, output_info, log_frequency)

     model = CTGAN()

     generator  = model.make_generator(sampler, transformer)

2023-04-19 15:32:52.137887: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


: 

: 

In [1]:
import tensorflow as tf
import numpy as np
batch_size = 5
embedding_dim = 128
fakez = tf.random.normal(shape=(batch_size, embedding_dim), mean= 0, stddev= 1)
condvec = sampler.sample_condvec(batch_size)

c1, m1, col, opt = condvec
c1 = tf.convert_to_tensor(np.array(c1))
c1 = tf.cast(c1, dtype=tf.float32)

m1 = tf.convert_to_tensor(np.array(m1))
m1 = tf.cast(m1, dtype=tf.int32)

fakez = tf.concat([fakez, c1], axis=1)

fake = generator(fakez)

2023-04-19 15:37:20.932327: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-04-19 15:37:21.833424: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-04-19 15:37:21.834245: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-04-19 15:37:21.858890: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 47.45GiB deviceMemoryBandwidth: 625.94GiB/s
2023-04-19 15:37:21.859163: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:af:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 47.45GiB deviceMemoryBandwidth: 625.94GiB/s
2023-04-19 15:3

NameError: name 'sampler' is not defined

In [104]:
batch_size = 5
condvec = sampler.sample_condvec(batch_size)
c1, m1, col, opt = condvec

In [105]:
c1

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0.

In [168]:
loss = []

In [169]:
labels = c1[:, 0:14]
logits=fake[:, 16:30]
tmp = tf.nn.softmax_cross_entropy_with_logits(labels,logits)
loss.append(tmp)


In [162]:
tmp

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([0.      , 3.166882, 0.      , 0.      , 0.      ], dtype=float32)>

In [170]:
labels = c1[:, 14:53]
logits=fake[:, 30:69]
tmp = tf.nn.softmax_cross_entropy_with_logits(labels,logits)
loss.append(tmp)


In [171]:
tmp

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)>

In [172]:
labels = c1[:, 53:74]
logits=fake[:, 69:90]
tmp = tf.nn.softmax_cross_entropy_with_logits(labels,logits)
loss.append(tmp)


In [173]:
tmp

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([2.6690714, 0.       , 2.6825294, 2.8406324, 3.0264661],
      dtype=float32)>

In [174]:
loss

[<tf.Tensor: shape=(5,), dtype=float32, numpy=array([0.      , 3.166882, 0.      , 0.      , 0.      ], dtype=float32)>,
 <tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)>,
 <tf.Tensor: shape=(5,), dtype=float32, numpy=
 array([2.6690714, 0.       , 2.6825294, 2.8406324, 3.0264661],
       dtype=float32)>]

In [175]:
loss = tf.stack(loss, axis=1)
loss

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[0.       , 0.       , 2.6690714],
       [3.166882 , 0.       , 0.       ],
       [0.       , 0.       , 2.6825294],
       [0.       , 0.       , 2.8406324],
       [0.       , 0.       , 3.0264661]], dtype=float32)>

In [155]:
m1

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [156]:
loss *m1

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[   0.     ,    0.     ,  521.37866],
       [ 303.00626,    0.     ,    0.     ],
       [   0.     ,    0.     ,  781.2013 ],
       [   0.     ,    0.     ,  707.9749 ],
       [   0.     ,    0.     , 1028.0239 ]], dtype=float32)>

In [157]:
tf.reduce_mean(loss * m1)

<tf.Tensor: shape=(), dtype=float32, numpy=222.77232>